Example 02

arg($\mathsf{a}$, 0.5). (**0.5**) \
arg($\mathsf{b}$, 0.5). (**0.25**) \
arg($\mathsf{c}$, 0.5). (**0.75**) \
arg($\mathsf{d}$, 0.5). (**0.5**) \
arg($\mathsf{e}$, 1). (**1**)

supp($\mathsf{b}$, $\mathsf{a}$). \
att($\mathsf{c}$, $\mathsf{a}$). \
supp($\mathsf{d}$, $\mathsf{a}$). \
att($\mathsf{e}$, $\mathsf{b}$). \
supp($\mathsf{e}$, $\mathsf{c}$).

In [41]:
%%capture
# install dependencies
try:
    import sympy
except:
    import sys
    !{sys.executable} -m pip install sympy

In [42]:
import numpy as np
import sympy as sp
sp.init_printing()

a, b, c, d, e = sp.symbols('a b c d e')  # arguments

energy, initial = sp.symbols('energy initial')
impact = lambda energy : sp.Max(0, energy)**2 / (1 + sp.Max(0, energy)**2)
combine = lambda initial, energy : initial - initial*impact(-energy) + (1-initial)*impact(energy)

E_a, E_b, E_c, E_d, E_e = sp.symbols('E_a E_b E_c E_d E_e')  # E_x - symbolic variable for the energy of argument x

In [43]:
E_e = 0  # e has no children
fs_e = combine(e, E_e)  # fs_x - symbolic variable for the final strength of argument x
fs_e

e

In [44]:
e_initial = e.evalf(subs={e: 1.0}, chop=True)  # x_initial - initial strength of argument x
e_initial

1.00000000000000

In [45]:
e_final = fs_e.evalf(subs={e: e_initial})  # x_final - final strength of argument x
e_final

1.00000000000000

In [46]:
E_d = 0  # d has no children
fs_d = combine(d, E_d)
fs_d

d

In [47]:
d_initial = d.evalf(subs={d: 0.5}, chop=True)
d_initial

0.500000000000000

In [48]:
d_final = fs_d.evalf(subs={d: d_initial, e: e_initial})
d_final

0.500000000000000

In [49]:
E_c = fs_e  # e is the only supporter of c
fs_c = combine(c, E_c)
fs_c

                 2                     2
     c⋅Max(0, -e)     (1 - c)⋅Max(0, e) 
c - ─────────────── + ──────────────────
              2                  2      
    Max(0, -e)  + 1     Max(0, e)  + 1  

In [50]:
c_initial = c.evalf(subs={c: 0.5}, chop=True)
c_initial

0.500000000000000

In [51]:
c_final = fs_c.evalf(subs={c: c_initial, e: e_initial})
c_final

0.750000000000000

In [52]:
E_b = -fs_e  # e is the only attacker of b
fs_b = combine(b, E_b)
fs_b

                2                      2
     b⋅Max(0, e)     (1 - b)⋅Max(0, -e) 
b - ────────────── + ───────────────────
             2                   2      
    Max(0, e)  + 1     Max(0, -e)  + 1  

In [53]:
b_initial = b.evalf(subs={b: 0.5}, chop=True)
b_initial

0.500000000000000

In [54]:
b_final = fs_b.evalf(subs={b: b_initial, e: e_initial})
b_final

0.250000000000000

In [55]:
E_a = fs_b + fs_d - fs_c
fs_a = combine(a, E_a)
fs_a

                                                                              
     a⋅Max(0, -b + b*Max(0, e)**2/(Max(0, e)**2 + 1) + c - c*Max(0, -e)**2/(Ma
a - ──────────────────────────────────────────────────────────────────────────
                                                                              
    Max(0, -b + b*Max(0, e)**2/(Max(0, e)**2 + 1) + c - c*Max(0, -e)**2/(Max(0

                                                                              
x(0, -e)**2 + 1) - d - (1 - b)*Max(0, -e)**2/(Max(0, -e)**2 + 1) + (1 - c)*Max
──────────────────────────────────────────────────────────────────────────────
                                                                              
, -e)**2 + 1) - d - (1 - b)*Max(0, -e)**2/(Max(0, -e)**2 + 1) + (1 - c)*Max(0,

                             2                                                
(0, e)**2/(Max(0, e)**2 + 1))     (1 - a)⋅Max(0, b - b*Max(0, e)**2/(Max(0, e)
─────────────────────────────── + ────────────────

In [56]:
a_initial = a.evalf(subs={a: 0.5}, chop=True)
a_initial

0.500000000000000

In [57]:
a_final = fs_a.evalf(subs={a: a_initial, b: b_initial, c: c_initial, d: d_initial, e: e_initial})
a_final

0.500000000000000

We can compute the contributions to $\mathsf{a}$ according to the gradient saliency method, evaluating them at the point of initial strengths. Note that in this case the final strength of $\mathsf{a}$ does not change from the initial one, and all the contributions are actually 0. E.g.

In [58]:
pdfs_a_b = fs_a.diff(b)  # pdf_x - partial derivative of f wrt x
pdfs_a_b.evalf(subs = {a: a_initial, b: b_initial, c: c_initial, d: d_initial, e: e_initial})

0

Other contributions need not be nil, e.g. from $\mathsf{e}$ to $\mathsf{b}$:

In [59]:
pdfs_b_e = fs_b.diff(e)
pdfs_b_e.evalf(subs = {a: a_initial, b: b_initial, c: c_initial, d: d_initial, e: e_initial})

-0.250000000000000